In [2]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Download and Load in the datasets

In [3]:
import pandas as pd
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="notmooodoo9/TrumpsTruthSocialPosts", 
    repo_type="dataset", 
    local_dir="datasets/trump_posts_and_comments", 
    local_dir_use_symlinks=False)

snapshot_download(
    repo_id="FamilyLinks/btc-price-1m-2017-2025", 
    repo_type="dataset", 
    local_dir="datasets/btc_prices", 
    local_dir_use_symlinks=False)

btc_df = pd.read_parquet("datasets/btc_prices/BTC_Raw_Micro_Macro_1m.parquet")
posts_df = pd.read_csv("datasets/trump_posts_and_comments/truthsocial.posts[Trump-FROM-10-8-25].csv")
reader = pd.read_csv("datasets/trump_posts_and_comments/truthsocial.comments[Trump-FROM-10-8-25].csv", chunksize=100000)

/home/valdemar/Repos/trump-tweets-analysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/valdemar/Repos/trump-tweets-analysis/.venv/lib/python3.12/site-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_use_symlinks` argument is deprecated and ignored in `snapshot_download`. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Fetching 3 files: 100%|██████████| 3/3 [00:00<00:00, 1943.01it/s]


### Load all 31.8M comments in chunks, calculating sentiment and dropping the comment text

In [4]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

# Load all 31.8M comments and calculate sentiment in chunks
chunks = []
print("Starting optimized load of 31M comments")
for i, chunk in enumerate(reader):
    # Calculate sentiment for each comment in the chunk
    chunk['sentiment_vader'] = chunk['text'].apply(lambda x: sid.polarity_scores(str(x))['compound'] if pd.notna(x) else 0.0)
    
    # Dont save text, to not crash the PC, (running out of RAM otherwise)
    mini_chunk = chunk[['_id', 'reply_to', 'sentiment_vader']].copy()
    chunks.append(mini_chunk)
    
    if i % 50 == 0: 
        print(f"Processed {i * 100_000} comments...")

comments_df = pd.concat(chunks, ignore_index=True)
print(f"Loaded {len(comments_df)} relevant comments")
print(f"Loaded {len(posts_df)} relevant posts.")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/valdemar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Starting optimized load of 31M comments
Processed 0 comments...
Processed 5000000 comments...
Processed 10000000 comments...
Processed 15000000 comments...
Processed 20000000 comments...
Processed 25000000 comments...
Processed 30000000 comments...
Loaded 31864491 relevant comments
Loaded 18476 relevant posts.


### Normalize the datset to use UTC timezone

In [5]:
# Convert BTC timestamp, its already in UTC
btc_df['timestamp_utc'] = pd.to_datetime(btc_df['date'])
btc_df['timestamp_utc'] = btc_df['timestamp_utc'].dt.tz_localize('UTC')

# Truth Social runs on a fork of Mastodon. 
# Mastodon IDs are 64-bit integers generated using a "Snowflake" ID system.
# Top 48 bits represent the number of milliseconds since the UNIX epoch.

# Convert Truth social posts ids to UTC timestamps
posts_df['timestamp_utc'] = pd.to_datetime(posts_df['_id'].apply(lambda x: x >> 16), unit='ms').dt.tz_localize('UTC')
posts_df = posts_df.sort_values('timestamp_utc')

# Convert Truth social comments ids to UTC timestamps
comments_df['timestamp_utc'] = pd.to_datetime(comments_df['_id'].apply(lambda x: x >> 16), unit='ms').dt.tz_localize('UTC')
comments_df = comments_df.sort_values('timestamp_utc')

### Connect multiple levels of comments to original post

In [6]:
from IPython.display import display

# Init trump post id as direct parent, use Int64 to prevent Nan replies.
comments_df['trump_post_id'] = comments_df['reply_to'].astype('Int64')

# Hash map for fast lookup the parent ID
# Index = Comment ID, Value = Parent ID
parent_map = comments_df[['_id', 'reply_to']].drop_duplicates(subset='_id').set_index('_id')['reply_to'].astype('Int64')

# Iterate max up to 300 times to find all grandparents (trump posts)
# Basically a comment can reply to another comment, and so on
for i in range(300):
    # look up the grandparent for current parent, if current parent is a post => NA because not in parent_map
    next_step = comments_df['trump_post_id'].map(parent_map)
    
    # Count how many rows still point to a comment
    n_changes = next_step.count()
    print(f"Iteration {i+1}, updated number of comments: {n_changes} ")
    
    if n_changes == 0:
        print("Convergence reached, found all grand parents.")
        break
        
    # Update only the rows that found a new parent
    # .update ignores NA values in next step, so then we already found the grand parent
    comments_df['trump_post_id'].update(next_step)

print("BTC Data Head:")
display(btc_df[['timestamp_utc', 'open']].head())
print("\nPosts Data Head:")
display(posts_df[['timestamp_utc', 'text']].head())
print("\nComments Data Head:")
display(comments_df[['_id', 'timestamp_utc', 'sentiment_vader', 'trump_post_id']].head())

Iteration 1, updated number of comments: 8171348 


/tmp/ipykernel_9570/2970614398.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  comments_df['trump_post_id'].update(next_step)


Iteration 2, updated number of comments: 3820274 
Iteration 3, updated number of comments: 2570223 
Iteration 4, updated number of comments: 1927530 
Iteration 5, updated number of comments: 1538245 
Iteration 6, updated number of comments: 1263572 
Iteration 7, updated number of comments: 1065394 
Iteration 8, updated number of comments: 910725 
Iteration 9, updated number of comments: 790289 
Iteration 10, updated number of comments: 691143 
Iteration 11, updated number of comments: 610761 
Iteration 12, updated number of comments: 543441 
Iteration 13, updated number of comments: 487253 
Iteration 14, updated number of comments: 438744 
Iteration 15, updated number of comments: 397563 
Iteration 16, updated number of comments: 361442 
Iteration 17, updated number of comments: 329909 
Iteration 18, updated number of comments: 301935 
Iteration 19, updated number of comments: 277155 
Iteration 20, updated number of comments: 255201 
Iteration 21, updated number of comments: 235881 
It

,timestamp_utc,open
0,2017-08-17 04:00:00+00:00,4261.48
1,2017-08-17 04:01:00+00:00,4261.48
2,2017-08-17 04:02:00+00:00,4280.56
3,2017-08-17 04:03:00+00:00,4261.48
4,2017-08-17 04:04:00+00:00,4261.48



Posts Data Head:


,timestamp_utc,text
18475,2022-02-14 15:54:32.523000+00:00,Get Ready! Your favorite President will see yo...
18474,2022-04-28 21:29:28.112000+00:00,I’M BACK! #COVFEFE
18473,2022-04-29 22:45:26.489000+00:00,Thank you to all of the GREAT and BEAUTIFUL Am...
18472,2022-04-30 12:37:05.097000+00:00,RT @catturd2Joe Biden is going to fly gas guzz...
18471,2022-04-30 12:37:13.338000+00:00,@melmul



Comments Data Head:


,_id,timestamp_utc,sentiment_vader,trump_post_id
31825494,107797431608971120,2022-02-14 17:04:30.400000+00:00,0.6588,107797156496908384
31830460,107797465322979408,2022-02-14 17:13:04.835000+00:00,0.0000,107797156496908384
31830461,107797688089449616,2022-02-14 18:09:43.982000+00:00,0.8374,107797156496908384
31830595,107803525915377408,2022-02-15 18:54:22.136000+00:00,0.5255,107797156496908384
31830596,107803530278287120,2022-02-15 18:55:28.709000+00:00,-0.5046,107797156496908384


### Analyse BTC Price Movement for every trump posts

In [7]:
price_windows = {
    'return_30m': 30,
    'return_1h': 60,
    'return_2h': 120,
    'return_4h': 240
}

def get_multi_window_returns(post_time):
    results = {}
    
    # Find start price for each post time
    start_idx = btc_df['timestamp_utc'].searchsorted(post_time)
    if start_idx >= len(btc_df):
        return pd.Series({k: None for k in price_windows})
        
    start_price = btc_df.iloc[start_idx]['open']
    
    # Calculate returns for each window
    for name, minutes in price_windows.items():
        end_time = post_time + pd.Timedelta(minutes=minutes)
        end_idx = btc_df['timestamp_utc'].searchsorted(end_time)
        
        if end_idx < len(btc_df):
            end_price = btc_df.iloc[end_idx]['open']
            results[name] = (end_price - start_price) / start_price
        else:
            results[name] = None
            
    return pd.Series(results)

# Calculate returns for multiple windows, adding them as new columns in trump posts dataframe
posts_df[list(price_windows.keys())] = posts_df['timestamp_utc'].apply(get_multi_window_returns)

print("BTC price returns windows from posts head:")
display(posts_df.head())

BTC price returns windows from posts head:


,_id,owner,text,timestamp_utc,return_30m,return_1h,return_2h,return_4h
18475,107797156496908384,107780257626128496,Get Ready! Your favorite President will see yo...,2022-02-14 15:54:32.523000+00:00,0.003583,0.001959,-0.000141,-0.011884
18474,108211822140637680,107780257626128496,I’M BACK! #COVFEFE,2022-04-28 21:29:28.112000+00:00,-0.000361,-0.004347,-0.002254,0.000982
18473,108217783188791696,107780257626128496,Thank you to all of the GREAT and BEAUTIFUL Am...,2022-04-29 22:45:26.489000+00:00,0.002005,0.002299,-0.000149,0.003185
18472,108221053343991936,107780257626128496,RT @catturd2Joe Biden is going to fly gas guzz...,2022-04-30 12:37:05.097000+00:00,0.000315,-0.000258,-0.005320,0.000634
18471,108221053884053056,107780257626128496,@melmul,2022-04-30 12:37:13.338000+00:00,0.000315,-0.000258,-0.005320,0.000634


### Sentiment analysis

In [8]:
# Apply sentiment analysis to posts
posts_df['sentiment_vader'] = posts_df['text'].apply(lambda x: sid.polarity_scores(str(x))['compound'] if pd.notna(x) else 0.0)

# Merge post timestamp to comments to calculate time delta
comments_with_post_time = comments_df.merge(
    posts_df[['_id', 'timestamp_utc']], 
    left_on='trump_post_id', 
    right_on='_id', 
    suffixes=('', '_post') # This creates 'timestamp_utc_post'
)

# Calculate time difference in minutes between comment and its Trump post
comments_with_post_time['minutes_after_post'] = (
    comments_with_post_time['timestamp_utc'] - comments_with_post_time['timestamp_utc_post']
).dt.total_seconds() / 60

# Calculate average comment sentiment per Trump post for different time windows
minutes_window = [30, 60, 120, 240] 
for minutes in minutes_window:
    window_mask = (comments_with_post_time['minutes_after_post'] >= 0) & \
                  (comments_with_post_time['minutes_after_post'] <= minutes)
    
    window_comments = comments_with_post_time[window_mask]
    
    avg_sentiment = window_comments.groupby('trump_post_id')['sentiment_vader'].mean()
    avg_sentiment.name = f'avg_comment_sentiment_{minutes}m'
    
    posts_df = posts_df.merge(avg_sentiment, left_on='_id', right_index=True, how='left')

print("Post & Comment sentiments head:")
display(posts_df.head())

Post & Comment sentiments head:


,_id,owner,text,timestamp_utc,return_30m,return_1h,return_2h,return_4h,sentiment_vader,avg_comment_sentiment_30m,avg_comment_sentiment_60m,avg_comment_sentiment_120m,avg_comment_sentiment_240m
18475,107797156496908384,107780257626128496,Get Ready! Your favorite President will see yo...,2022-02-14 15:54:32.523000+00:00,0.003583,0.001959,-0.000141,-0.011884,0.7256,NaN,NaN,0.329400,0.498733
18474,108211822140637680,107780257626128496,I’M BACK! #COVFEFE,2022-04-28 21:29:28.112000+00:00,-0.000361,-0.004347,-0.002254,0.000982,0.0000,0.314743,0.320155,0.319895,0.320966
18473,108217783188791696,107780257626128496,Thank you to all of the GREAT and BEAUTIFUL Am...,2022-04-29 22:45:26.489000+00:00,0.002005,0.002299,-0.000149,0.003185,0.9600,0.405271,0.407812,0.409905,0.414320
18472,108221053343991936,107780257626128496,RT @catturd2Joe Biden is going to fly gas guzz...,2022-04-30 12:37:05.097000+00:00,0.000315,-0.000258,-0.005320,0.000634,-0.6428,NaN,NaN,NaN,NaN
18471,108221053884053056,107780257626128496,@melmul,2022-04-30 12:37:13.338000+00:00,0.000315,-0.000258,-0.005320,0.000634,0.0000,0.290516,0.276212,0.283446,0.277534


### Keywords analysis

In [11]:
keywords = ['btc', "bitcoin", 'crypto', "cryptocurrency", 'tariff', "tariffs", 
            'dollar', 'usa', "china", "market", "economy", "inflation",
            "coin", "currency", "blockchain", "invest", "fed", "bull", "bear", 
            "stock", "trade", "exchange", "token", 
            "halving", "fiat", "payment", "volatility", "wealth", "financial"]
results = []

# Filter for post 2024 election (nov 5 2024)
# Because of relevant change in political finance influence
election_2024 = pd.Timestamp("2024-11-05").tz_localize("UTC")
posts_df = posts_df[posts_df['timestamp_utc'] >= election_2024]
comments_df = comments_df[comments_df['timestamp_utc'] >= election_2024]

# BTC return after posts Baseline 
baseline_30m_returns = posts_df['return_30m'].mean() * 100
baseline_1h_returns = posts_df['return_1h'].mean() * 100
baseline_2h_returns = posts_df['return_2h'].mean() * 100
baseline_4h_returns = posts_df['return_4h'].mean() * 100

# Trump Baseline Sentiment 
baseline_sentiment_posts = posts_df['sentiment_vader'].mean()
baseline_sentiment_comments = comments_df['sentiment_vader'].mean()

print(f"Baseline Returns (All Posts): 30m={baseline_30m_returns:.4f}%, 1h={baseline_1h_returns:.4f}%, 2h={baseline_2h_returns:.4f}%, 4h={baseline_4h_returns:.4f}%")
print(f"Baseline Sentiment (All Posts): {baseline_sentiment_posts:.4f}")
print(f"Baseline Sentiment (All Comments): {baseline_sentiment_comments:.4f}")

for kw in keywords:
    # Filter Posts containing the keywords
    # Regex with word boundaries to match whole words only
    subset = posts_df[posts_df['text'].str.lower().str.contains(rf"\b{kw}\b", regex=True, na=False)]

    if not subset.empty:
        # Average BTC returns after posts with the keyword
        avg_30m = subset['return_30m'].mean() * 100
        avg_1h = subset['return_1h'].mean() * 100
        avg_2h = subset['return_2h'].mean() * 100
        avg_4h = subset['return_4h'].mean() * 100
        
        # The average sentiment for the posts with the keyword
        avg_post_sentiment = subset['sentiment_vader'].mean()

        # How many comments does a post with this keyword usually get?
        kw_comments_count = len(comments_df[comments_df['trump_post_id'].isin(subset['_id'])])
        avg_comments_per_post = kw_comments_count / len(subset)
        
        # Average Comment Sentiment for posts containing the keyword
        all_com_senti_30m = subset['avg_comment_sentiment_30m'].mean()
        all_com_senti_1h = subset['avg_comment_sentiment_60m'].mean()
        all_com_senti_2h = subset['avg_comment_sentiment_120m'].mean()
        
        results.append({
            'keyword': kw,
            'post_count': len(subset),
            'avg_comments': round(avg_comments_per_post, 1), 

            # Return Metrics
            '30m_%': avg_30m,
            '1h_%': avg_1h,
            '2h_%': avg_2h,
            '4h_%': avg_4h,

            # Comparison vs Baseline
            '30m_vs_base': avg_30m - baseline_30m_returns,
            '1h_vs_base': avg_1h - baseline_1h_returns,
            '2h_vs_base': avg_2h - baseline_2h_returns,
            '4h_vs_base': avg_4h - baseline_4h_returns,
            
            # Post Sentiment Metrics
            'avg_post_senti': avg_post_sentiment,
            'avg_post_senti_vs_base': avg_post_sentiment - baseline_sentiment_posts, # Is this topic happier than usual?
            
            # Average Comment Sentiment
            'all_com_senti_30m': all_com_senti_30m,
            'all_com_senti_1h': all_com_senti_1h,
            'all_com_senti_2h': all_com_senti_2h      
        })

pd.set_option('display.max_colwidth', None)
# Display Results, sort them by the 1h btc return
results_df = pd.DataFrame(results).sort_values('1h_%', ascending=False) 
display(results_df.round(4))

print(f"\nTop 5 Keywords with BEST 1h return:")
for kw in results_df.head(5)['keyword'].tolist():
    print(f"keyword {kw}")
    subset = posts_df[posts_df['text'].str.lower().str.contains(rf"\b{kw}\b", regex=True, na=False)]
    ex_posts = subset.sort_values('return_1h', ascending=False).head(1)    
    display(ex_posts)

print(f"\nTop 5 Keywords with WORST 1h return:")
for kw in results_df.tail(5)['keyword'].tolist():
    print(f"keyword {kw}")
    subset = posts_df[posts_df['text'].str.lower().str.contains(rf"\b{kw}\b", regex=True, na=False)]
    ex_posts = subset.sort_values('return_1h', ascending=True).head(1)    
    display(ex_posts)

Baseline Returns (All Posts): 30m=0.0019%, 1h=0.0126%, 2h=0.0359%, 4h=0.0992%
Baseline Sentiment (All Posts): 0.3197
Baseline Sentiment (All Comments): 0.0211


,keyword,post_count,avg_comments,30m_%,1h_%,2h_%,4h_%,30m_vs_base,1h_vs_base,2h_vs_base,4h_vs_base,avg_post_senti,avg_post_senti_vs_base,all_com_senti_30m,all_com_senti_1h,all_com_senti_2h
0,btc,1,2448.0,2.9261,2.1660,1.4119,2.7601,2.9243,2.1534,1.3760,2.6609,0.8221,0.5024,0.1261,0.1241,0.1195
3,cryptocurrency,1,1840.0,0.3908,1.4771,0.5131,0.7112,0.3890,1.4645,0.4772,0.6120,0.9432,0.6235,0.1373,0.1388,0.1334
13,currency,5,5381.8,1.3635,1.3393,1.2350,1.9839,1.3616,1.3267,1.1990,1.8847,-0.0129,-0.3326,0.0748,0.0691,0.0660
2,crypto,9,1525.1,0.0979,0.7717,1.3180,1.4611,0.0960,0.7591,1.2820,1.3619,0.7303,0.4106,0.0670,0.0622,0.0625
20,token,2,1113.0,0.3570,0.6330,1.0046,1.2009,0.3551,0.6203,0.9686,1.1017,0.9458,0.6261,0.0880,0.0910,0.0804
1,bitcoin,4,1377.8,0.5957,0.3194,0.1675,0.5385,0.5939,0.3068,0.1316,0.4392,0.2828,-0.0370,0.0852,0.0863,0.0962
19,exchange,5,2176.8,0.1297,0.1523,0.1341,0.5811,0.1278,0.1397,0.0982,0.4819,0.4548,0.1351,0.0559,0.0475,0.0389
8,china,76,3348.7,0.0701,0.1201,0.1068,0.1470,0.0682,0.1075,0.0709,0.0478,0.3399,0.0202,0.0424,0.0370,0.0284
18,trade,111,2679.2,0.0917,0.0964,0.0326,0.2031,0.0898,0.0837,-0.0033,0.1039,0.5600,0.2402,0.1073,0.1014,0.0993
10,economy,143,1903.5,0.0228,0.0391,0.1306,0.1695,0.0209,0.0265,0.0947,0.0703,0.7408,0.4211,0.0623,0.0494,0.0376



Top 5 Keywords with BEST 1h return:
keyword btc


,_id,owner,text,timestamp_utc,return_30m,return_1h,return_2h,return_4h,sentiment_vader,avg_comment_sentiment_30m,avg_comment_sentiment_60m,avg_comment_sentiment_120m,avg_comment_sentiment_240m
2112,114093946326587360,107780257626128496,"RT: obviously, BTC and ETH, as other valuable Cryptocurrencies, will be the heart of the Reserve. I also love Bitcoin and Ethereum!",2025-03-02 17:11:00.305000+00:00,0.029261,0.02166,0.014119,0.027601,0.8221,0.126074,0.124055,0.119542,0.123911


keyword cryptocurrency


...

,_id,owner,text,timestamp_utc,return_30m,return_1h,return_2h,return_4h,sentiment_vader,avg_comment_sentiment_30m,avg_comment_sentiment_60m,avg_comment_sentiment_120m,avg_comment_sentiment_240m
808,114857617089281824,107780257626128496,"Consumer Prices LOW. Bring down the Fed Rate, NOW!!!",2025-07-15 14:02:31.386000+00:00,-0.006263,-0.018707,-0.015167,-0.007643,-0.5732,-0.051001,-0.056441,-0.078244,-0.080334


keyword bear


,_id,owner,text,timestamp_utc,return_30m,return_1h,return_2h,return_4h,sentiment_vader,avg_comment_sentiment_30m,avg_comment_sentiment_60m,avg_comment_sentiment_120m,avg_comment_sentiment_240m
1690,114338375079660272,107780257626128496,"Thank you very much to all of the Record Setting Attendees! We are spending a lot of time, effort, and money on making The Kennedy Center the spectacular venue that it was originally meant to be. Bear with us, it’s happening faster than expected. The crowds are fantastic, nothing will keep them away!",2025-04-14 21:12:27.089000+00:00,-0.00258,-0.0016,-0.00121,-0.000242,0.7707,0.046181,-0.005813,-0.060117,-0.034505


keyword payment


,_id,owner,text,timestamp_utc,return_30m,return_1h,return_2h,return_4h,sentiment_vader,avg_comment_sentiment_30m,avg_comment_sentiment_60m,avg_comment_sentiment_120m,avg_comment_sentiment_240m
2278,113991086021543776,107780257626128496,"DOGE has found massive amounts of FRAUD, WASTE, INCOMPETENCE, AND ABUSE, but even knowing this, a highly political, activist Judge wants us to immediately make payment, anyway. In other words pay, even though you know the payment was fraudulently requested to be made. DOGE caught them - The Judge just doesn’t care. It doesn’t make sense!!!",2025-02-12 13:12:16.608000+00:00,-0.014162,-0.009446,-0.011836,0.011013,-0.7712,-0.113883,-0.120864,-0.122994,-0.132082
